# **Reinforcement Learning**

**Equipo:**
* Mayra Medrano (100%)
* Jyns Ordóñez (100%)
* Diego Antonio Rosario Palomino (100%)
* Xiomara Mayela Siche Eusebio (100%)

### **Objetivo:**

Comprender de forma integral el proceso de modelado y resolución de un MDP sencillo (diseñado por ti mismo) mediante el algoritmo de **Value Iteration**, obteniendo la función de valor óptima $V_*(s)$ y la política óptima $\pi_*$

### **Tareas:**

1. Recordar la definición formal de un **Proceso de Decisión de Markov (MDP)**.
2. Formular un problema realista como un MDP: *estados*, *acciones*, *transiciones* y *recompensas*.
3. Implementar el algoritmo de **Value Iteration** para obtener la función de valor óptima $V_*(s)$.
4. Derivar una política óptima $\pi_*$ a partir de $V_*(s)$ y discutir los resultados.

### **Entregables**

1. **Canvas**: Notebook completo con tu MDP definido, código ejecutado y reflexiones respondidas.
2. **Foro**: Publicar su MDP y compartir sus reflexiones.
3. Se calificará la **claridad** del modelado, la **correctitud** del algoritmo y la **profundidad** de la discusión (no la complejidad del MDP).

## **0. Contexto del problema**

En esta práctica **tú** modelarás, programarás y resolverás un MDP a partir de la siguiente narrativa:

> Eres el diseñador/a de un **robot mensajero** que se desplaza por un pequeño pasillo de 4 casillas numeradas \(0,1,2,3\).  
> * El robot **siempre empieza** en la casilla 0.  
> * En la casilla 3 hay una estación de entrega que otorga **recompensa +1** y **termina** el episodio.  
> * Cada vez que el robot ejecuta una acción que **no** llega a la estación, recibe **recompensa −0.1** (costo de energía).  
> * **Acciones disponibles** (en cada estado que no sea terminal):  
>   • **"avanza"**: intenta moverse +1 casilla.  
>   • **"retrocede"**: intenta moverse −1 casilla (si ya está en 0 se queda).  
>   • **"espera"**: permanece en la misma casilla.  
> * Debido a fallos de motor, **cada acción tiene un 10 % de probabilidad de no ejecutarse** y deja al robot en la misma casilla (sin recompensa extra).  
> * El algoritmo debe usar un **descuento $\gamma = 0.9$**.

Tu **tarea** es **formalizar** esa narrativa como un MDP (listas de `states`, `actions`, `recompensas`, y matriz `P` de transiciones estocásticas) y luego implementar *Value Iteration* para hallar $V_*(s)$ y la política óptima $\pi_*$.

## **1. Define tu MDP**
Abajo encontrarás un esqueleto con TODOs: rellena las listas de estados y acciones, y completa la función `build_P()` que devuelve el diccionario `P` con la estructura:

```python
P[state][action] = [(probabilidad, estado_siguiente, recompensa), ...]
```

In [1]:
# 1️⃣ ESTADOS
states = [0, 1, 2, 3]
terminal_states = [3]

# 2️⃣ ACCIONES
actions = ["avanza", "retrocede", "espera"]

# 3️⃣ TRANSICIONES

def build_P():
    """Devuelve un diccionario P[s][a] = [(p, s', r), ...]."""
    P = {s: {a: [] for a in actions} for s in states}
    
    for s in states:
        if s in terminal_states:
            continue  # No hay acciones desde estados terminales
        
        for a in actions:
            # Por defecto, 10% de probabilidad de falla (se queda en el mismo lugar)
            fail_prob = 0.1
            fail_state = s
            fail_reward = -0.1

            if a == "avanza":
                success_state = min(s + 1, 3)
            elif a == "retrocede":
                success_state = max(s - 1, 0)
            elif a == "espera":
                success_state = s
            else:
                continue

            if success_state == 3:
                success_reward = 1
            else:
                success_reward = -0.1

            # 90% éxito + 10% falla
            P[s][a].append((0.9, success_state, success_reward))
            P[s][a].append((0.1, fail_state, fail_reward))

    return P

P = build_P()

# Ejemplo de transición
from pprint import pprint
print("Ejemplo de P[0][\"avanza\"] (debería existir):")
pprint(P[0]["avanza"])

Ejemplo de P[0]["avanza"] (debería existir):
[(0.9, 1, -0.1), (0.1, 0, -0.1)]


## 2. Algoritmo: Value Iteration
Recordemos la ecuación de Bellman para la función de valor óptima:
$$
    V_*(s) = \max_{a \in \mathcal{A}} \sum_{s'} p(s'\,|\,s,a)\,\big[r(s,a,s') + \gamma \, V_*(s')\big].
$$
El algoritmo itera hasta que el cambio máximo entre dos iteraciones es menor que un umbral $\theta$.

In [2]:
import numpy as np
from tabulate import tabulate

gamma = 0.9   # factor de descuento
theta = 1e-6  # umbral de convergencia

V = {s: 0.0 for s in states}

iteration = 0
while True:
    delta = 0.0
    for s in states:
        if s in terminal_states:
            continue  # no actualizamos estados terminales
        v = V[s]
        # Calcula el valor de cada acción
        action_values = []
        for a in actions:
            q_sa = sum(p * (r + gamma * V[s_]) for (p, s_, r) in P[s][a])
            action_values.append(q_sa)
        # Encuentra el máximo valor entre todas las acciones
        V[s] = max(action_values)
        delta = max(delta, abs(v - V[s]))
    iteration += 1
    if delta < theta:
        break

print(f"Converged in {iteration} iterations\n")
print(tabulate([[s, round(V[s], 4)] for s in states], headers=["Estado", "V*(s)"]))

Converged in 11 iterations

  Estado    V*(s)
--------  -------
       0   0.5672
       1   0.7607
       2   0.978
       3   0


## **3. Derivación de la política óptima**
Una vez que cuentas con $V_*(s)$, basta con seleccionar en cada estado la acción que maximiza el retorno esperado:

In [3]:
def greedy_policy(state: int):
    if state in terminal_states:
        return None
    best_a, best_val = None, -np.inf
    for a in actions:
        q_sa = sum(p * (r + gamma * V[s_]) for (p, s_, r) in P[state][a])
        if q_sa > best_val:
            best_val = q_sa
            best_a = a
    return best_a

policy = {s: greedy_policy(s) for s in states}

print("Política óptima:")
print(tabulate([[s, policy[s]] for s in states], headers=["Estado", "π*(s)"]))

Política óptima:
  Estado  π*(s)
--------  -------
       0  avanza
       1  avanza
       2  avanza
       3


## **4. Preguntas para reflexionar**
1. **Modelado:** ¿Cómo representa tu MDP la probabilidad de fallo? ¿Por qué asignaste −0.1 como costo por paso?
2. **Convergencia:** ¿Cuántas iteraciones tomó? Prueba con $\gamma = 0.5$ y 0.99, ¿qué varía y por qué?
3. **Comparación:** Ajusta la probabilidad de fallo al 20 % y observa cómo cambia $V$ y la política. Explica intuitivamente.
4. **Escalabilidad:** ¿Qué obstáculos ves para aplicar Value Iteration cuando el número de casillas crece a 100?

1. Modelado: ¿Cómo representa tu MDP la probabilidad de fallo? ¿Por qué asignaste −0.1 como costo por paso?

La probabilidad de fallo está representada en el diccionario P[s][a] como una transición que lleva al mismo estado con una probabilidad del 10%. Es decir, cada acción tiene 90% de éxito y 10% de que no pase nada.

Se asignó un costo de −0.1 por paso para incentivar al agente a llegar al objetivo lo más rápido posible. Si no hubiera penalidad, "esperar" podría parecer tan bueno como avanzar. El costo por paso ayuda a modelar eficiencia temporal o energética.

2. **Convergencia:** ¿Cuántas iteraciones tomó? Prueba con $\gamma = 0.5$ y 0.99, ¿qué varía y por qué?

In [4]:
import numpy as np
from tabulate import tabulate

# 1. Definimos estados y acciones
states = [0, 1, 2, 3]
terminal_states = [3]
actions = ["avanza", "retrocede", "espera"]

# 2. Transiciones con 10% de fallo
def build_P():
    P = {s: {a: [] for a in actions} for s in states}
    for s in states:
        if s in terminal_states:
            continue
        for a in actions:
            fail_prob = 0.1
            success_prob = 0.9
            fail_state = s
            fail_reward = -0.1

            if a == "avanza":
                success_state = min(s + 1, 3)
            elif a == "retrocede":
                success_state = max(s - 1, 0)
            elif a == "espera":
                success_state = s

            success_reward = 1 if success_state == 3 else -0.1

            P[s][a].append((success_prob, success_state, success_reward))
            P[s][a].append((fail_prob, fail_state, fail_reward))
    return P

P = build_P()

# 3. Función de Value Iteration
def value_iteration(gamma, theta=1e-6):
    V = {s: 0.0 for s in states}
    iteration = 0
    while True:
        delta = 0.0
        for s in states:
            if s in terminal_states:
                continue
            v = V[s]
            action_values = []
            for a in actions:
                q_sa = sum(p * (r + gamma * V[s_]) for (p, s_, r) in P[s][a])
                action_values.append(q_sa)
            V[s] = max(action_values)
            delta = max(delta, abs(v - V[s]))
        iteration += 1
        if delta < theta:
            break
    return V, iteration

# 4. Ejecutar para gamma = 0.5 y 0.99
for gamma in [0.5, 0.99]:
    V, iteration = value_iteration(gamma)
    print(f"\n Convergió con γ = {gamma} en {iteration} iteraciones")
    print(tabulate([[s, round(V[s], 4)] for s in states], headers=["Estado", f"V*(s) γ={gamma}"]))


 Convergió con γ = 0.5 en 9 iteraciones
  Estado    V*(s) γ=0.5
--------  -------------
       0         0.0551
       1         0.3385
       2         0.9368
       3         0

 Convergió con γ = 0.99 en 11 iteraciones
  Estado    V*(s) γ=0.99
--------  --------------
       0          0.7452
       1          0.8658
       2          0.9878
       3          0


2. **Convergencia:** Al ejecutar el algoritmo de *Value Iteration* con distintos valores de $\gamma$, obtuve los siguientes resultados:

- Con $\gamma = 0.5$, el algoritmo convergió en 9 iteraciones.
- Con $\gamma = 0.99$, el algoritmo convergió en 11 iteraciones, igual que con el valor original de $\gamma = 0.9$.

Esto ocurre porque el parámetro $\gamma$ (factor de descuento) determina cuánto valora el agente las recompensas futuras.  
- Con **$\gamma = 0.5$**, el agente da poca importancia al futuro, por lo que las recompensas se propagan menos a través de los estados. Esto hace que el valor converja más rápido, pero también que los valores óptimos sean más bajos, especialmente en los estados alejados del objetivo.
- Con $\gamma = 0.99$, el agente considera fuertemente las recompensas futuras, lo que provoca una propagación más lenta de los valores y, por tanto, más iteraciones para converger. Sin embargo, los valores $V^*(s)$ son mucho mayores, ya que reflejan una visión a largo plazo.

3. **Comparación:** Ajusta la probabilidad de fallo al 20 % y observa cómo cambia $V$ y la política. Explica intuitivamente.

In [5]:
import numpy as np
from tabulate import tabulate

# 1. Estados y acciones
states = [0, 1, 2, 3]
terminal_states = [3]
actions = ["avanza", "retrocede", "espera"]

# 2. Función para construir transiciones con probabilidad de fallo ajustable
def build_P(fail_prob):
    success_prob = 1 - fail_prob
    P = {s: {a: [] for a in actions} for s in states}
    for s in states:
        if s in terminal_states:
            continue
        for a in actions:
            fail_state = s
            fail_reward = -0.1

            if a == "avanza":
                success_state = min(s + 1, 3)
            elif a == "retrocede":
                success_state = max(s - 1, 0)
            elif a == "espera":
                success_state = s

            success_reward = 1 if success_state == 3 else -0.1

            P[s][a].append((success_prob, success_state, success_reward))
            P[s][a].append((fail_prob, fail_state, fail_reward))
    return P

# 3. Value Iteration
def value_iteration(P, gamma=0.9, theta=1e-6):
    V = {s: 0.0 for s in states}
    iteration = 0
    while True:
        delta = 0.0
        for s in states:
            if s in terminal_states:
                continue
            v = V[s]
            action_values = []
            for a in actions:
                q_sa = sum(p * (r + gamma * V[s_]) for (p, s_, r) in P[s][a])
                action_values.append(q_sa)
            V[s] = max(action_values)
            delta = max(delta, abs(v - V[s]))
        iteration += 1
        if delta < theta:
            break
    return V, iteration

# 4. Política óptima
def greedy_policy(V, P, state: int, gamma=0.9):
    if state in terminal_states:
        return None
    best_a, best_val = None, -np.inf
    for a in actions:
        q_sa = sum(p * (r + gamma * V[s_]) for (p, s_, r) in P[state][a])
        if q_sa > best_val:
            best_val = q_sa
            best_a = a
    return best_a

# 5. Ejecutamos para fallo del 10% y 20%
P_10 = build_P(0.1)
P_20 = build_P(0.2)

V_10, _ = value_iteration(P_10)
V_20, _ = value_iteration(P_20)

# 6. Imprimimos la tabla comparativa
tabla = []
for s in states:
    fila = [
        s,
        round(V_10[s], 4),
        round(V_20[s], 4),
        greedy_policy(V_10, P_10, s),
        greedy_policy(V_20, P_20, s)
    ]
    tabla.append(fila)

print(tabulate(tabla, headers=["Estado", "V*(s) fallo 10%", "V*(s) fallo 20%", "π*(s) fallo 10%", "π*(s) fallo 20%"]))

  Estado    V*(s) fallo 10%    V*(s) fallo 20%  π*(s) fallo 10%    π*(s) fallo 20%
--------  -----------------  -----------------  -----------------  -----------------
       0             0.5672             0.5043  avanza             avanza
       1             0.7607             0.7133  avanza             avanza
       2             0.978              0.9512  avanza             avanza
       3             0                  0


**4. Escalabilidad:**  
Cuando el número de casillas aumenta, por ejemplo de 4 a 100, aplicar Value Iteration se vuelve más costoso en términos de tiempo y memoria. Los principales obstáculos son:

- **Tamaño del espacio de estados:**  
  El algoritmo de Value Iteration realiza actualizaciones sobre todos los estados en cada iteración. Si hay 100 casillas, esto implica actualizar 100 estados en cada paso.

- **Complejidad computacional:**  
  Por cada estado, se evalúan todas las acciones posibles. Para un número de estados $|S|$ y acciones $|A|$, la complejidad por iteración es $\mathcal{O}(|S||A|)$. En un entorno con 100 estados y 3 acciones, eso ya implica 300 evaluaciones por iteración.

- **Propagación de valores lenta:**  
  Cuando $\gamma$ es alto (como 0.9 o 0.99), los valores de los estados lejanos al objetivo tardan más en ajustarse, lo cual aumenta el número de iteraciones necesarias para converger.

- **Memoria para representar transiciones:**  
  El diccionario de transiciones $P[s][a]$ crece proporcionalmente al número de estados y acciones. Con 100 estados, el tamaño del modelo puede volverse considerable, especialmente si hay incertidumbre en las acciones.
